# Preliminary test on a Single Bank

In [1]:
import os
import pdftables_api
import glob2
import tabula
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
pd.set_option("max_colwidth", 3000)
pd.set_option("max_rows", 2000)
pd.set_option("max_columns", 2000)

# 1. Statement extraction

In [54]:
DATA_FOLDER = './statements/'
MY_API_KEY = ''

In [55]:
glob2.glob(os.path.join(DATA_FOLDER, "*"))

['./statements/224912180637606770417463100932.pdf',
 './statements/Account Statement (1).pdf',
 './statements/420 STATEMENT.pdf',
 './statements/CUSTOMER_STATEMENT_OF_ACCT_ONLINE_F12[NWOLU].pdf',
 './statements/0163041071-(205154861110).pdf',
 './statements/account_stmt_24Apr2021_24Jun2021.pdf',
 './statements/68197656963758929248882067034.pdf',
 './statements/OFFER LETTER MR FITZGERALD Signed 01072021.pdf',
 './statements/output.csv',
 './statements/420 PROPERTIES LIMITED.pdf',
 './statements/fav Ajo.pdf',
 './statements/statement_adeshina iliasu.pdf',
 './statements/224912180637606770417463100932output.csv',
 './statements/68197656963758929248882067034output.csv']

In [56]:
def process_bank_statements(b_statements_gt_bank, out_format ='csv'):
    '''
    Method to transform to csv a list of Bank statements paths
    '''
    
    n_statements = len(b_statements_gt_bank)
    for bk_st in tqdm(b_statements_gt_bank):
        inp = bk_st
        out = bk_st.replace(".pdf","output.csv")
        tabula.convert_into(inp, out, output_format=out_format)
        
    print( f"{n_statements} bank_statements processed!")

In [57]:
bank_statements_gt_bank = ["./statements/224912180637606770417463100932.pdf",
                          "./statements/68197656963758929248882067034.pdf"]

In [58]:
process_bank_statements(bank_statements_gt_bank, out_format ='csv')

100%|██████████| 2/2 [00:03<00:00,  1.76s/it]

2 bank_statements processed!


In [59]:
def bank_statement_to_dataframe(b_statement_gt_bank, mult=True, pages_all='all'):
    '''
    Function to transform to dataframe a Bank statement pdf
    
    b_statement_gt_bank: a statement path of a Bank statement pdf
    mult: consider various tables in the document
    pages_all: keep all pages if 'all'
    '''
    df = tabula.read_pdf(b_statement_gt_bank,
                         multiple_tables=mult,
                         pages=pages_all)
    return df

In [317]:
df = tabula.read_pdf("./statements/224912180637606770417463100932.pdf", 
                     multiple_tables=True,
                    pages='all')

In [318]:
b_statement = bank_statements_gt_bank[1]
customer = bank_statement_to_dataframe(b_statement, mult=True, pages_all='all')

In [329]:
type_account = type_account.T.reset_index().T.reset_index(drop=True)
type_account.columns = ["PI", "Detail"]

In [332]:
account_customer_description = type_account[type_account["PI"]== "Account Type"]["Detail"].values[0]

In [62]:
class GT_BankStatement():
    
    def __init__(self, b_statement_gt_bank, bank_id='GT_Bank'):
        '''
        initializes the Gt_bank statement class
        '''
        # a bankstatement has 2 input attributes (for now)
        # bank_id & statement pdf path
        
        self.bank_id = bank_id
        self.customer_bank_statement = bank_statement_to_dataframe(b_statement_gt_bank, mult=True, pages_all='all')
        self.df = self.customer_bank_statement.copy()
        
    def check_table(self):
        '''
        Store the list of columns names of all extracted dataframes
        '''
        # store the columns names for debug
        self.df_cols = [b_statement_dataframe.columns for b_statement_dataframe in self.df]
        return self.df_cols
    
    def black_list_narrow_tables(self, verbose = False):
        '''
        removes the tables that do not contain transaction data
        '''
        
        # each pdf can extract tables
        # (some tables will be without headers)
        # We want to recreate the header (which matches table 1)
        
        reformatted_dfs = [self.df[k].T.reset_index().T for k in range(1,len(self.df),1)]
        self.finalized_dfs = []
        self.blacklisted_tables_idxs = []
        
        for idx,r_df in enumerate(reformatted_dfs): 
            try:
                r_df.columns = df[1].columns
                r_df.index = range(r_df.shape[0])
                r_df = r_df.replace(regex=[r'^Unnamed:..$'], value=np.nan)
                self.finalized_dfs.append(r_df)
            except:
                # if it fails, it is because it is a table of another type
                # they will be blacklisted and kept for potential processing
                self.blacklisted_tables_idxs.append(idx)
        if verbose:
            return self.finalized_dfs
    
    def combine_dataframes(self):
        '''
        combines dataframes that contain financial data
        '''
        # combine all whitelisted dataframes (=transactions)
        self.master_df = pd.concat(self.finalized_dfs[:]).reset_index(drop=True)
        
    def clean_dataframe(self):
        '''
        simple data cleaning to remove non informative cells
        '''
        # extra cleaning
        self.master_df = self.master_df[self.master_df['Remarks']!= 'Remarks']
        self.master_df = self.master_df[self.master_df['Remarks']!= 'Balance as at Last Transaction.']
        self.master_df = self.master_df[self.master_df["Trans. Date"] != 'Trans. Date']
        self.master_df = self.master_df.reset_index(drop=True)
        
    def clean_transactions(self):
        '''
        keep only the transactions with dates and add one fictious date for easier processing
        
        '''
        self.transaction_not_null = self.master_df[~self.master_df["Trans. Date"].isna()].copy()
        self.index_list_of_transaction = list(self.transaction_not_null.index)
        self.transaction_not_null.loc[self.master_df.index.max() + 1,'Trans. Date'] = '99-Apr-9999'
        
    def postprocess(self, verbose = False):
        '''
        reconstruct the financial operations which overflow to the next line in 1 single text
        '''
        
        # all the indexes of the transaction with dates
        self.index_with_dates = self.transaction_not_null.index
        
        self.descr = {}
        for step in self.index_with_dates:
            self.descr[str(step)] = []

        step = 0    

        for idx, step in enumerate(self.index_with_dates):
            if idx < len(self.index_with_dates)-1:
                for ind in range(self.index_with_dates[idx], self.index_with_dates[idx+1], 1):
                    if ind < self.index_with_dates[idx+1]: 
                        if str(self.master_df.loc[ind, 'Remarks']) != 'nan':
                            self.descr[str(step)] += [str(self.master_df.loc[ind, 'Remarks'])]
            else:
                for ind in range(self.index_with_dates[idx], self.master_df.shape[0], 1):
                    if str(self.master_df.loc[ind, 'Remarks']) != 'nan':
                        self.descr[str(step)] += [str(self.master_df.loc[ind, 'Remarks'])]
                        
        for key in self.descr.keys():
            self.descr[key] = (''.join(self.descr[key])).replace('\r',' ')
        if verbose:
            return self.descr
    
    def recombined_dataframe(self, verbose= False):
        '''
        reconstruct the final dataset with all original transaction information plus the annotations
        '''
        # Dataframe of the transactions
        annotations = pd.DataFrame.from_dict(self.descr,  orient='index', columns=['Remarks_processed'])
        self.dataset_recombined = pd.concat([self.transaction_not_null.reset_index(drop=True), annotations.reset_index(drop=True)], axis=1)
        if verbose:
            return self.dataset_recombined
        

In [63]:
customer1 = GT_BankStatement("./statements/224912180637606770417463100932.pdf")

In [64]:
customer1.black_list_narrow_tables()

In [65]:
customer1.combine_dataframes()

In [66]:
customer1.clean_dataframe()

In [67]:
customer1.clean_transactions()

In [68]:
customer1.postprocess()

In [69]:
customer1.recombined_dataframe()

In [70]:
customer1.dataset_recombined

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks,Remarks_processed
0,01-Apr-2021,01-Apr-2021,'0NIPG,"6,180.00",NaN,118.78,E- CHANNELS,NIBSS Instant Payment Outward,NIBSS Instant Payment Outward000013210401074110000065685369 viaGTWORLD TO LAGASEF VENTURES LTD/26.875/REF:GW2249121800000000618021040107f
1,01-Apr-2021,01-Apr-2021,'0NIPG,25.00,NaN,93.78,E- CHANNELS,COMMISSION 000013210401074110000065685369,COMMISSION 000013210401074110000065685369NIP TRANSFER COMMISSION FOR000013210401074110000065685369 viaGTWORLD TO LAGASEF VENTURES LTDReF:GW224912180000000061802104010739
2,01-Apr-2021,01-Apr-2021.1,'0NIPG,1.87,NaN,91.91,E- CHANNELS,VALUE ADDED TAX\r000013210401074110000065685369 VAT ON NIP\rTRANSFER FOR\r000013210401074110000065685369 via\rGTWORLD TO LAGASEF VENTURES LTD\rReF:GW224912180000000061802104010739,VALUE ADDED TAX 000013210401074110000065685369 VAT ON NIP TRANSFER FOR 000013210401074110000065685369 via GTWORLD TO LAGASEF VENTURES LTD ReF:GW224912180000000061802104010739
3,01-Apr-2021,01-Apr-2021,'0NIPT,NaN,"12,800.00","12,891.91",E- CHANNELS,"TRANSFER BETWEEN CUSTOMERS\r000014210401100626212783422352|Fundstrf for\rADELANA PAUL OLANREWAJU (0456561509) by\rAgent ,ISMAIL WALE GISANRIN\r(1428457311).|AGEB7649779306959803987||14284\r57311 REF:000014210401100626212783422352","TRANSFER BETWEEN CUSTOMERS 000014210401100626212783422352|Fundstrf for ADELANA PAUL OLANREWAJU (0456561509) by Agent ,ISMAIL WALE GISANRIN (1428457311).|AGEB7649779306959803987||14284 57311 REF:000014210401100626212783422352"
4,01-Apr-2021,01-Apr-2021,'0NIPG,"12,690.00",NaN,201.91,E- CHANNELS,NIBSS Instant Payment Outward\r000013210401101641000065959536 via\rGTWORLD TO LAGASEF VENTURES LTD\r/26.875/REF:GW2249121800000001269021040110\rf,NIBSS Instant Payment Outward 000013210401101641000065959536 via GTWORLD TO LAGASEF VENTURES LTD /26.875/REF:GW2249121800000001269021040110 f
5,01-Apr-2021,01-Apr-2021,'0NIPG,25.00,NaN,176.91,E- CHANNELS,COMMISSION 000013210401101641000065959536\rNIP TRANSFER COMMISSION FOR\r000013210401101641000065959536 via\rGTWORLD TO LAGASEF VENTURES LTD\rReF:GW224912180000000126902104011014,COMMISSION 000013210401101641000065959536 NIP TRANSFER COMMISSION FOR 000013210401101641000065959536 via GTWORLD TO LAGASEF VENTURES LTD ReF:GW224912180000000126902104011014
6,01-Apr-2021,01-Apr-2021,'0NIPG,1.87,NaN,175.04,E- CHANNELS,VALUE ADDED TAX\r000013210401101641000065959536 VAT ON NIP\rTRANSFER FOR\r000013210401101641000065959536 via\rGTWORLD TO LAGASEF VENTURES LTD\rReF:GW224912180000000126902104011014,VALUE ADDED TAX 000013210401101641000065959536 VAT ON NIP TRANSFER FOR 000013210401101641000065959536 via GTWORLD TO LAGASEF VENTURES LTD ReF:GW224912180000000126902104011014
7,01-Apr-2021,01-Apr-2021,'0NIPT,NaN,"7,000.00","7,175.04",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS\r000003210401201509000793996528|QR\rQ10738609073 Trf To ADELANA PAUL\rOLANREWAJU|S62195789||0530116010\rREF:000003210401201509000793996528,TRANSFER BETWEEN CUSTOMERS 000003210401201509000793996528|QR Q10738609073 Trf To ADELANA PAUL OLANREWAJU|S62195789||0530116010 REF:000003210401201509000793996528
8,02-Apr-2021,02-Apr-2021,'0NIPG,"7,080.00",NaN,95.04,E- CHANNELS,NIBSS Instant Payment Outward\r000013210402053717000067430400 via\rGTWORLD TO LAGASEF VENTURES LTD\r/26.875/REF:GW2249121800000000708021040205\rf,NIBSS Instant Payment Outward 000013210402053717000067430400 via GTWORLD TO LAGASEF VENTURES LTD /26.875/REF:GW2249121800000000708021040205 f
9,02-Apr-2021,02-Apr-2021.1,'0NIPG,25.00,NaN,70.04,E- CHANNELS,COMMISSION 000013210402053717000067430400\rNIP TRANSFER COMMISSION FOR\r000013210402053717000067430400 via\rGTWORLD TO LAGASEF VENTURES LTD\rReF:GW224912180000000070802104020535,COMMISSION 000013210402053717000067430400 NIP TRANSFER COMMISSION FOR 000013210402053717000067430400 via GTWORLD TO LAGASEF VENTURES LTD ReF:GW224912180000000070802104020535


## 2. Features engineering (-> classifier)

In [71]:
class Dataset_Bank():
    def __init__(self, Data):
        self._data = Data
        
    def check_keyword(self, keyword):
        self._data["IS_A_"+ keyword] =  self._data["Remarks_processed"].str.contains(keyword, case=False)
    
    def filter_text(self):
        self._data["filtered_description"] = self._data["Remarks_processed"].str.findall('[^\d\W]+').str.join(sep =' ')
    
    def dest(self):
        self._data["filtered_dest"] = self._data["filtered_description"].str.upper()
        self._data["filtered_dest"] = self._data["filtered_dest"].str.split("TO").str[-1]

In [72]:
GT_Dataset = Dataset_Bank(customer1.dataset_recombined)

In [73]:
GT_Dataset.check_keyword("TRANSFER")
GT_Dataset.check_keyword("TRANSFER COMMISSION")
GT_Dataset.check_keyword("CONTRIBUTION")
GT_Dataset.check_keyword("TAX")
GT_Dataset.check_keyword("SALARY")
GT_Dataset.check_keyword("VAT")
GT_Dataset.check_keyword("WEB PURCHASE")
GT_Dataset.check_keyword("POS/WEB")
GT_Dataset.check_keyword("AIRTIME")
GT_Dataset.check_keyword("PURCHASE")
GT_Dataset.check_keyword("INSTANT PAYMENT")

GT_Dataset.check_keyword("CASH")
GT_Dataset.check_keyword("WITHDRAWAL")
GT_Dataset.check_keyword("ATM")
GT_Dataset.check_keyword("SMS ALERT CHARGE")
GT_Dataset.check_keyword("FAILED")
GT_Dataset.check_keyword("FUNDS")
GT_Dataset.check_keyword("LOAN")
GT_Dataset.check_keyword("INTEREST")
GT_Dataset.check_keyword("GTWORLD")

In [74]:
GT_Dataset.filter_text()

In [75]:
GT_Dataset.dest()

In [76]:
GT_Dataset._data.head()

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks,Remarks_processed,IS_A_TRANSFER,IS_A_TRANSFER COMMISSION,IS_A_CONTRIBUTION,IS_A_TAX,IS_A_SALARY,IS_A_VAT,IS_A_WEB PURCHASE,IS_A_POS/WEB,IS_A_AIRTIME,IS_A_PURCHASE,IS_A_INSTANT PAYMENT,IS_A_CASH,IS_A_WITHDRAWAL,IS_A_ATM,IS_A_SMS ALERT CHARGE,IS_A_FAILED,IS_A_FUNDS,IS_A_LOAN,IS_A_INTEREST,IS_A_GTWORLD,filtered_description,filtered_dest
0,01-Apr-2021,01-Apr-2021,'0NIPG,"6,180.00",NaN,118.78,E- CHANNELS,NIBSS Instant Payment Outward,NIBSS Instant Payment Outward000013210401074110000065685369 viaGTWORLD TO LAGASEF VENTURES LTD/26.875/REF:GW2249121800000000618021040107f,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,NIBSS Instant Payment Outward viaGTWORLD TO LAGASEF VENTURES LTD REF GW f,LAGASEF VENTURES LTD REF GW F
1,01-Apr-2021,01-Apr-2021,'0NIPG,25.00,NaN,93.78,E- CHANNELS,COMMISSION 000013210401074110000065685369,COMMISSION 000013210401074110000065685369NIP TRANSFER COMMISSION FOR000013210401074110000065685369 viaGTWORLD TO LAGASEF VENTURES LTDReF:GW224912180000000061802104010739,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,COMMISSION NIP TRANSFER COMMISSION FOR viaGTWORLD TO LAGASEF VENTURES LTDReF GW,LAGASEF VENTURES LTDREF GW
2,01-Apr-2021,01-Apr-2021.1,'0NIPG,1.87,NaN,91.91,E- CHANNELS,VALUE ADDED TAX\r000013210401074110000065685369 VAT ON NIP\rTRANSFER FOR\r000013210401074110000065685369 via\rGTWORLD TO LAGASEF VENTURES LTD\rReF:GW224912180000000061802104010739,VALUE ADDED TAX 000013210401074110000065685369 VAT ON NIP TRANSFER FOR 000013210401074110000065685369 via GTWORLD TO LAGASEF VENTURES LTD ReF:GW224912180000000061802104010739,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,VALUE ADDED TAX VAT ON NIP TRANSFER FOR via GTWORLD TO LAGASEF VENTURES LTD ReF GW,LAGASEF VENTURES LTD REF GW
3,01-Apr-2021,01-Apr-2021,'0NIPT,NaN,"12,800.00","12,891.91",E- CHANNELS,"TRANSFER BETWEEN CUSTOMERS\r000014210401100626212783422352|Fundstrf for\rADELANA PAUL OLANREWAJU (0456561509) by\rAgent ,ISMAIL WALE GISANRIN\r(1428457311).|AGEB7649779306959803987||14284\r57311 REF:000014210401100626212783422352","TRANSFER BETWEEN CUSTOMERS 000014210401100626212783422352|Fundstrf for ADELANA PAUL OLANREWAJU (0456561509) by Agent ,ISMAIL WALE GISANRIN (1428457311).|AGEB7649779306959803987||14284 57311 REF:000014210401100626212783422352",True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,TRANSFER BETWEEN CUSTOMERS Fundstrf for ADELANA PAUL OLANREWAJU by Agent ISMAIL WALE GISANRIN AGEB REF,MERS FUNDSTRF FOR ADELANA PAUL OLANREWAJU BY AGENT ISMAIL WALE GISANRIN AGEB REF
4,01-Apr-2021,01-Apr-2021,'0NIPG,"12,690.00",NaN,201.91,E- CHANNELS,NIBSS Instant Payment Outward\r000013210401101641000065959536 via\rGTWORLD TO LAGASEF VENTURES LTD\r/26.875/REF:GW2249121800000001269021040110\rf,NIBSS Instant Payment Outward 000013210401101641000065959536 via GTWORLD TO LAGASEF VENTURES LTD /26.875/REF:GW2249121800000001269021040110 f,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,NIBSS Instant Payment Outward via GTWORLD TO LAGASEF VENTURES LTD REF GW f,LAGASEF VENTURES LTD REF GW F


In [77]:
csv_to_annotate = GT_Dataset._data

In [78]:
csv_to_annotate.columns

Index(['Trans. Date', 'Value. Date', 'Reference', 'Debits', 'Credits',
       'Balance', 'Originating Branch', 'Remarks', 'Remarks_processed',
       'IS_A_TRANSFER', 'IS_A_TRANSFER COMMISSION', 'IS_A_CONTRIBUTION',
       'IS_A_TAX', 'IS_A_SALARY', 'IS_A_VAT', 'IS_A_WEB PURCHASE',
       'IS_A_POS/WEB', 'IS_A_AIRTIME', 'IS_A_PURCHASE', 'IS_A_INSTANT PAYMENT',
       'IS_A_CASH', 'IS_A_WITHDRAWAL', 'IS_A_ATM', 'IS_A_SMS ALERT CHARGE',
       'IS_A_FAILED', 'IS_A_FUNDS', 'IS_A_LOAN', 'IS_A_INTEREST',
       'IS_A_GTWORLD', 'filtered_description', 'filtered_dest'],
      dtype='object')

In [83]:
# to do check against a certain regexp
# unit tests

In [79]:
# fix the "date format" problem inside timestamp cell
# problem found : 02-Apr-2021.1
for c in csv_to_annotate.columns[:2]:
    csv_to_annotate[c] = csv_to_annotate[c].apply(lambda x : str(x).split(".")[0])

In [80]:
# fix the "\r" problem inside each cell

for c in csv_to_annotate.columns:
    try:
        csv_to_annotate[c] = csv_to_annotate[c].str.replace("\r"," ")
    except AttributeError:
        print(c)

IS_A_TRANSFER
IS_A_TRANSFER COMMISSION
IS_A_CONTRIBUTION
IS_A_TAX
IS_A_SALARY
IS_A_VAT
IS_A_WEB PURCHASE
IS_A_POS/WEB
IS_A_AIRTIME
IS_A_PURCHASE
IS_A_INSTANT PAYMENT
IS_A_CASH
IS_A_WITHDRAWAL
IS_A_ATM
IS_A_SMS ALERT CHARGE
IS_A_FAILED
IS_A_FUNDS
IS_A_LOAN
IS_A_INTEREST
IS_A_GTWORLD


In [81]:
csv_to_annotate

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks,Remarks_processed,IS_A_TRANSFER,IS_A_TRANSFER COMMISSION,IS_A_CONTRIBUTION,IS_A_TAX,IS_A_SALARY,IS_A_VAT,IS_A_WEB PURCHASE,IS_A_POS/WEB,IS_A_AIRTIME,IS_A_PURCHASE,IS_A_INSTANT PAYMENT,IS_A_CASH,IS_A_WITHDRAWAL,IS_A_ATM,IS_A_SMS ALERT CHARGE,IS_A_FAILED,IS_A_FUNDS,IS_A_LOAN,IS_A_INTEREST,IS_A_GTWORLD,filtered_description,filtered_dest
0,01-Apr-2021,01-Apr-2021,'0NIPG,"6,180.00",NaN,NaN,E- CHANNELS,NIBSS Instant Payment Outward,NIBSS Instant Payment Outward000013210401074110000065685369 viaGTWORLD TO LAGASEF VENTURES LTD/26.875/REF:GW2249121800000000618021040107f,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,NIBSS Instant Payment Outward viaGTWORLD TO LAGASEF VENTURES LTD REF GW f,LAGASEF VENTURES LTD REF GW F
1,01-Apr-2021,01-Apr-2021,'0NIPG,25.00,NaN,NaN,E- CHANNELS,COMMISSION 000013210401074110000065685369,COMMISSION 000013210401074110000065685369NIP TRANSFER COMMISSION FOR000013210401074110000065685369 viaGTWORLD TO LAGASEF VENTURES LTDReF:GW224912180000000061802104010739,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,COMMISSION NIP TRANSFER COMMISSION FOR viaGTWORLD TO LAGASEF VENTURES LTDReF GW,LAGASEF VENTURES LTDREF GW
2,01-Apr-2021,01-Apr-2021,'0NIPG,1.87,NaN,91.91,E- CHANNELS,VALUE ADDED TAX 000013210401074110000065685369 VAT ON NIP TRANSFER FOR 000013210401074110000065685369 via GTWORLD TO LAGASEF VENTURES LTD ReF:GW224912180000000061802104010739,VALUE ADDED TAX 000013210401074110000065685369 VAT ON NIP TRANSFER FOR 000013210401074110000065685369 via GTWORLD TO LAGASEF VENTURES LTD ReF:GW224912180000000061802104010739,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,VALUE ADDED TAX VAT ON NIP TRANSFER FOR via GTWORLD TO LAGASEF VENTURES LTD ReF GW,LAGASEF VENTURES LTD REF GW
3,01-Apr-2021,01-Apr-2021,'0NIPT,NaN,"12,800.00","12,891.91",E- CHANNELS,"TRANSFER BETWEEN CUSTOMERS 000014210401100626212783422352|Fundstrf for ADELANA PAUL OLANREWAJU (0456561509) by Agent ,ISMAIL WALE GISANRIN (1428457311).|AGEB7649779306959803987||14284 57311 REF:000014210401100626212783422352","TRANSFER BETWEEN CUSTOMERS 000014210401100626212783422352|Fundstrf for ADELANA PAUL OLANREWAJU (0456561509) by Agent ,ISMAIL WALE GISANRIN (1428457311).|AGEB7649779306959803987||14284 57311 REF:000014210401100626212783422352",True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,TRANSFER BETWEEN CUSTOMERS Fundstrf for ADELANA PAUL OLANREWAJU by Agent ISMAIL WALE GISANRIN AGEB REF,MERS FUNDSTRF FOR ADELANA PAUL OLANREWAJU BY AGENT ISMAIL WALE GISANRIN AGEB REF
4,01-Apr-2021,01-Apr-2021,'0NIPG,"12,690.00",NaN,201.91,E- CHANNELS,NIBSS Instant Payment Outward 000013210401101641000065959536 via GTWORLD TO LAGASEF VENTURES LTD /26.875/REF:GW2249121800000001269021040110 f,NIBSS Instant Payment Outward 000013210401101641000065959536 via GTWORLD TO LAGASEF VENTURES LTD /26.875/REF:GW2249121800000001269021040110 f,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,NIBSS Instant Payment Outward via GTWORLD TO LAGASEF VENTURES LTD REF GW f,LAGASEF VENTURES LTD REF GW F
5,01-Apr-2021,01-Apr-2021,'0NIPG,25.00,NaN,176.91,E- CHANNELS,COMMISSION 000013210401101641000065959536 NIP TRANSFER COMMISSION FOR 000013210401101641000065959536 via GTWORLD TO LAGASEF VENTURES LTD ReF:GW224912180000000126902104011014,COMMISSION 000013210401101641000065959536 NIP TRANSFER COMMISSION FOR 000013210401101641000065959536 via GTWORLD TO LAGASEF VENTURES LTD ReF:GW224912180000000126902104011014,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,COMMISSION NIP TRANSFER COMMISSION FOR via GTWORLD TO LAGASEF 

In [154]:
csv_to_annotate.to_csv("./first_data.csv", index= None, sep = ";",encoding='utf-8-sig')

In [155]:
df =pd.read_csv("/Users/assansanogo/Downloads/first_data-first_data.csv")

In [156]:
df = df.iloc[:-1,:]


In [271]:
comm_index = df[df["Remarks"].str.contains("COMM").values].index
tax_index = df[df["Remarks"].str.contains("TAX").values].index
salary_index = df[df["Remarks"].str.contains("SALARY").values].index
loan_index = df[df["Remarks"].str.contains("LOAN").values].index

In [272]:
df.loc[comm_index,"Class"]="COMM"
df.loc[tax_index,"Class"]="TAX"
df.loc[salary_index,"Class"]="SALARY"
df.loc[salary_index,"Class"]="LOAN"

In [277]:
loan = df[df["Remarks"].str.contains("LOAN").values]

In [280]:
loan

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Class,Remarks,Remarks_processed,IS_A_TRANSFER,IS_A_TRANSFER COMMISSION,IS_A_CONTRIBUTION,IS_A_TAX,IS_A_SALARY,IS_A_VAT,IS_A_WEB PURCHASE,IS_A_POS/WEB,IS_A_AIRTIME,IS_A_PURCHASE,IS_A_INSTANT PAYMENT,IS_A_CASH,IS_A_WITHDRAWAL,IS_A_ATM,IS_A_SMS ALERT CHARGE,IS_A_FAILED,IS_A_FUNDS,IS_A_LOAN,IS_A_INTEREST,IS_A_GTWORLD,filtered_description,filtered_dest
345,16-Jun-2021,16-Jun-2021,00,"1,321.71",NaN,935.26,CATHOLIC MISSION,LOAN,MATURED LOAN INSTALLMENT Loan Repayment1,MATURED LOAN INSTALLMENT Loan Repayment1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,MATURED LOAN INSTALLMENT Loan Repayment,MATURED LOAN INSTALLMENT LOAN REPAYMENT
448,30-Apr-2021,30-Apr-2021,41210003862,"8,353.83",NaN,"-7,321.80",CATHOLIC MISSION,LOAN,MATURED LOAN INSTALLMENT,MATURED LOAN INSTALLMENT,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,MATURED LOAN INSTALLMENT,MATURED LOAN INSTALLMENT
449,30-Apr-2021,30-Apr-2021,41210001673,"10,907.89",NaN,"-18,229.69",CATHOLIC MISSION,LOAN,MATURED LOAN INSTALLMENT,MATURED LOAN INSTALLMENT,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,MATURED LOAN INSTALLMENT,MATURED LOAN INSTALLMENT
456,31-May-2021,31-May-2021,41210003863,"8,353.83",NaN,"-8,231.62",CATHOLIC MISSION,LOAN,MATURED LOAN INSTALLMENT,MATURED LOAN INSTALLMENT,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,MATURED LOAN INSTALLMENT,MATURED LOAN INSTALLMENT
457,31-May-2021,31-May-2021,41210001674,"10,907.89",NaN,"-19,139.51",CATHOLIC MISSION,LOAN,MATURED LOAN INSTALLMENT,MATURED LOAN INSTALLMENT,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,MATURED LOAN INSTALLMENT,MATURED LOAN INSTALLMENT
458,16-Jun-2021,16-Jun-2021,00,NaN,"1,321.71","-17,817.80",CATHOLIC MISSION,LOAN,MATURED LOAN INSTALLMENT Loan Repayment1,MATURED LOAN INSTALLMENT Loan Repayment1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,MATURED LOAN INSTALLMENT Loan Repayment,MATURED LOAN INSTALLMENT LOAN REPAYMENT


In [295]:
loan.loc[:]["Trans. Date"] = pd.to_datetime(loan["Trans. Date"], errors='coerce')

In [296]:
loan_summary = loan.loc[:,["Trans. Date", "Credits", "Debits"]].copy()

In [297]:
loan_summary.set_index('Trans. Date', inplace=True)

In [298]:
loan_summary.index = pd.to_datetime(loan_summary.index)

In [299]:
loan_summary["Debits"].fillna("0", inplace=True)
loan_summary["Credits"].fillna("0", inplace=True)

In [300]:
for cat in ["Credits", "Debits"]:
    loan_summary[cat]= loan_summary[cat].str.replace(",","").astype('float')


In [301]:
loan_summary

,Credits,Debits
Trans. Date,,
2021-06-16,0.00,1321.71
2021-04-30,0.00,8353.83
2021-04-30,0.00,10907.89
2021-05-31,0.00,8353.83
2021-05-31,0.00,10907.89
2021-06-16,1321.71,0.00


In [311]:
df_loan_summary = loan_summary.groupby(pd.Grouper(level='Trans. Date',freq='M')).sum()

In [312]:
n_loans_summary = loan_summary.groupby(pd.Grouper(level='Trans. Date',freq='M')).count()

In [313]:
n_loans_summary

,Credits,Debits
Trans. Date,,
2021-04-30,2,2
2021-05-31,2,2
2021-06-30,2,2


In [306]:
df_loan_summary["effective"] = df_loan_summary["Debits"]-df_loan_summary["Credits"]

In [307]:
df_loan_summary

,Credits,Debits,effective
Trans. Date,,,
2021-04-30,0.00,19261.72,19261.72
2021-05-31,0.00,19261.72,19261.72
2021-06-30,1321.71,1321.71,0.00


In [252]:
salary = df[df["Remarks"].str.contains("SALARY").values]

### 3. Analytics

In [253]:
# salary

In [254]:
salary.loc[:]["Trans. Date"] = pd.to_datetime(salary["Trans. Date"], errors='coerce')

In [255]:
salary.loc[:,["Trans. Date", "Credits"]]

,Trans. Date,Credits
441,28-Apr-2021,"52,605.90"
442,28-Apr-2021,"10,000.00"
451,28-May-2021,"52,605.90"
452,28-May-2021,"10,000.00"
461,28-Jun-2021,"10,000.00"
462,28-Jun-2021,"52,605.90"


In [256]:
salary

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Class,Remarks,Remarks_processed,IS_A_TRANSFER,IS_A_TRANSFER COMMISSION,IS_A_CONTRIBUTION,IS_A_TAX,IS_A_SALARY,IS_A_VAT,IS_A_WEB PURCHASE,IS_A_POS/WEB,IS_A_AIRTIME,IS_A_PURCHASE,IS_A_INSTANT PAYMENT,IS_A_CASH,IS_A_WITHDRAWAL,IS_A_ATM,IS_A_SMS ALERT CHARGE,IS_A_FAILED,IS_A_FUNDS,IS_A_LOAN,IS_A_INTEREST,IS_A_GTWORLD,filtered_description,filtered_dest
441,28-Apr-2021,28-Apr-2021,66126644GAPS,NaN,"52,605.90","36,632.03",E- CHANNELS,SALARY,MTHLY SALARY via GAPS 112121953 GTBank,MTHLY SALARY via GAPS 112121953 GTBankSalary for April 2021 from STRESERT SERVICESLIMITED to ADELANA PAUL OLANREWAJUREF:0001701329112121953000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS GTBankSalary for April from STRESERT SERVICESLIMITED to ADELANA PAUL OLANREWAJUREF,ADELANA PAUL OLANREWAJUREF
442,28-Apr-2021,28-Apr-2021,66128260GAPS,NaN,"10,000.00","46,632.03",E- CHANNELS,SALARY,MTHLY SALARY via GAPS 112123898 GTBank Bonus for April 2021 from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329112123898000000,MTHLY SALARY via GAPS 112123898 GTBank Bonus for April 2021 from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329112123898000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS GTBank Bonus for April from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF,ADELANA PAUL OLANREWAJU REF
451,28-May-2021,28-May-2021,69548935GAPK,NaN,"52,605.90","34,876.21",E- CHANNELS,SALARY,MTHLY SALARY via GAPS 113722179 May 2021 Salary for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329113722179000000,MTHLY SALARY via GAPS 113722179 May 2021 Salary for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329113722179000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS May Salary for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF,ADELANA PAUL OLANREWAJU REF
452,28-May-2021,28-May-2021,69549381GAPS,NaN,"10,000.00","44,876.21",E- CHANNELS,SALARY,MTHLY SALARY via GAPS 113722651 May 2021 Bonus for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329113722651000000,MTHLY SALARY via GAPS 113722651 May 2021 Bonus for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329113722651000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS May Bonus for GTBank Staff from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF,ADELANA PAUL OLANREWAJU REF
461,28-Jun-2021,28-Jun-2021,72865794GAPS,NaN,"10,000.00","-5,106.93",E- CHANNELS,SALARY,MTHLY SALARY via GAPS 115442885 GTBank Bonus for June 2021 from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329115442885000000,MTHLY SALARY via GAPS 115442885 GTBank Bonus for June 2021 from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329115442885000000,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,MTHLY SALARY via GAPS GTBank Bonus for June from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF,ADELANA PAUL OLANREWAJU REF
462,28-Jun-2021,28-Jun-2021,72867714GAPS,NaN,"52,605.90","47,498.97",E- CHANNELS,SALARY,MTHLY SALARY via GAPS 115445208 GTBank June 2021 Staff Salary from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329115445208000000,MTHLY SALARY via GAPS 115445208 GTBank June 2021 Staff Salary from STRESERT SERVICES LIMITED to ADELANA PAUL OLANREWAJU REF: 0001701329115445208000000,False,False,False,False,True,False,False,False,False,False,False,False,Fals

In [257]:
salary.set_index('Trans. Date', inplace=True)

In [258]:
salary.index = pd.to_datetime(salary.index)

In [270]:
number_of_payments = len(set(list(salary.index)))
paid_month = [el.month for el in set(list(salary.index))]
paid_days = [el.day for el in set(list(salary.index))]

paid_day = np.mean(paid_days)  

variance_in_paid_days = np.std(paid_days) 

0.0

In [265]:
salary_numeric = salary.loc[:,["Debits","Credits"]]

In [243]:
salary_numeric["Credits"]= salary_numeric["Credits"].str.replace(",","").astype('float')

In [246]:
df_salary = salary_numeric.groupby(pd.Grouper(level='Trans. Date',freq='M')).sum()

In [248]:
df_salary.mean()

Credits    62605.9
dtype: float64

In [308]:
def salary_bracket(salary_dollars):
    if salary_dollars > 50000 and salary_dollars < 200000:
        bracket = "LOW_EARNER"
    elif salary_dollars > 210000 and salary_dollars < 500000:
        bracket = "LOW_MED_EARNER"
    elif salary_dollars > 500000 and salary_dollars < 1000000:  
        bracket = "MED_EARNER"
    elif salary_dollars > 1000000 and salary_dollars < 3000000: 
        bracket = "HIGH_MED_EARNER"
    elif salary_dollars > 3000000 and salary_dollars < 5000000:
        bracket = "HIGH_EARNER"
    elif salary_dollars > 5000000:
        bracket = "TOP_EARNER"
    return bracket

In [310]:
DTI = {
"LOW_EARNER": 0.33,
"LOW_MED_EARNER":0.35,
"MED_EARNER":0.40,
"HIGH_MED_EARNER":0.45,
"HIGH_EARNER": 0.5,
"TOP_EARNER": 0.55
}

## 3. Classifier Model

## 4. Credit score extraction

### a. Credit score (final score)

In [392]:
l_credit_scores = glob2.glob('./credit_score/*')

In [393]:
credit_report_godwin = l_credit_scores[0]

In [394]:
TOP = 253
LEFT = 23
WIDTH = 280
HEIGHT = 28
PDF_WIDTH = 612
PDF_HEIGHT = 792



df = tabula.read_pdf(credit_report_godwin, 
                     multiple_tables=True,
                     area=[100*TOP/PDF_HEIGHT,
                           100*LEFT/PDF_WIDTH,
                           100*(TOP+ HEIGHT)/PDF_HEIGHT,
                           100*(LEFT+WIDTH)/PDF_WIDTH],
                     relative_area=True,
                    pages='1')

In [395]:
df[0]

,SCORE DATE,SCORE,CREDIT SCORE CLASS
0,05/07/2021,718,MEDIUM RISK


### b. Score summary breakdown

In [344]:
# repayment history
# total amount owed
# types of credits in use


TOP = 196
LEFT = 244
WIDTH = 112
HEIGHT = 54
PDF_WIDTH = 612
PDF_HEIGHT = 792



df = tabula.read_pdf(credit_report_godwin, 
                     multiple_tables=True,
                     area=[100*TOP/PDF_HEIGHT,
                           100*LEFT/PDF_WIDTH,
                           100*(TOP+ HEIGHT)/PDF_HEIGHT,
                           100*(LEFT+WIDTH)/PDF_WIDTH],
                     relative_area=True,
                    pages='1')

score_summary_breakdown_1 = df[0].T.reset_index(drop=True).T

In [361]:
# length of credit history
# number of credits



TOP = 180
LEFT = 356
WIDTH = 116
HEIGHT = 53
PDF_WIDTH = 612
PDF_HEIGHT = 792



df = tabula.read_pdf(credit_report_godwin, 
                     multiple_tables=True,
                     area=[100*TOP/PDF_HEIGHT,
                           100*LEFT/PDF_WIDTH,
                           100*(TOP+ HEIGHT)/PDF_HEIGHT,
                           100*(LEFT+WIDTH)/PDF_WIDTH],
                     relative_area=True,
                    pages='1')

score_summary_breakdown_2 = df[0].T.reset_index(drop=True).T

In [362]:
score_summary_breakdown = pd.concat([score_summary_breakdown_1,score_summary_breakdown_2], axis=0).T

In [363]:
new_header = [el.strip() for el in score_summary_breakdown.iloc[0]]
score_summary = score_summary_breakdown[1:]
score_summary.columns = new_header

In [369]:
score_summary

,Total Amount Owed:,Types of Credit(s) in Use:,Length of Credit History:,Number of Credits:
1,72/165,40/55,62/83,55/55


In [387]:
score_total_amount_owned = score_summary['Total Amount Owed:'].str.split("/").str[0].astype('float')
score_total_type_of_credits_in_use = score_summary['Types of Credit(s) in Use:'].str.split("/").str[0].astype('float')
score_total_length_of_credit_history = score_summary['Length of Credit History:'].str.split("/").str[0].astype('float')
score_total_number_of_credits = score_summary['Number of Credits:'].str.split("/").str[0].astype('float')

In [388]:
score_total_amount_owned_max = score_summary['Total Amount Owed:'].str.split("/").str[1].astype('float')
score_total_type_of_credits_in_use_max = score_summary['Types of Credit(s) in Use:'].str.split("/").str[1].astype('float')
score_total_length_of_credit_history_max = score_summary['Length of Credit History:'].str.split("/").str[1].astype('float')
score_total_number_of_credits_max = score_summary['Number of Credits:'].str.split("/").str[1].astype('float')

In [391]:
pd.DataFrame([score_total_amount_owned/score_total_amount_owned_max,
score_total_type_of_credits_in_use/score_total_type_of_credits_in_use_max,
score_total_length_of_credit_history/score_total_length_of_credit_history_max,
score_total_number_of_credits/score_total_number_of_credits_max]).T

,Total Amount Owed:,Types of Credit(s) in Use:,Length of Credit History:,Number of Credits:
1,0.436364,0.727273,0.746988,1.0


___________

In [34]:
# each pdf can extract tables
# (those tables will be without headers)

In [37]:
# this first table has the correct header
df[1].columns

Index(['Trans. Date', 'Value. Date', 'Reference', 'Debits', 'Credits',
       'Balance', 'Originating Branch', 'Remarks'],
      dtype='object')

In [38]:
# this second table DOES NOT have the correct header
df[2].columns

Index(['07-Dec-2020', '07-Dec-2020.1', ''0GWTR', '50,000.00', 'Unnamed: 0',
       '310,944.19', 'E- CHANNELS',
       'TRANSFER BETWEEN CUSTOMERS Via GTWorld\rEneke   REF:681976569000000500002012070913\rfrom ASHAFA IBRAHIM ABDULLAHI  to  SALAMI,\rABUBAKAR'],
      dtype='object')

In [39]:
# We want to recreate the header (which matches table 1)

In [40]:
reformatted_dfs = [df[k].T.reset_index().T for k in range(1,len(df),1)]
finalized_dfs = []
for idx,r_df in enumerate(reformatted_dfs): 
    try:
        r_df.columns = df[1].columns
        r_df.index = range(r_df.shape[0])
        r_df = r_df.replace(regex=[r'^Unnamed:..$'], value=np.nan)
        finalized_dfs.append(r_df)
    except:
        # if it fails, it is because it is a table of another type
        print(idx)

145


In [41]:
# Visualize the first table of the finalized dataframes
finalized_dfs[0]

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks
0,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks
1,07-Dec-2020,07-Dec-2020,'0GAPS,NaN,"308,750.00","278,851.80",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS via GAPS
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,com on Babawuro A 13M sales 104164162 0 from
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NETCONSTRUCT NIGERIA LIMITED to ASHAFA
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBRAHIM ABDULLAHI
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REF: 0005628626104164162000000
6,07-Dec-2020,07-Dec-2020,'0GAPS,NaN,"82,092.39","360,944.19",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS via GAPS
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,com on Ekene Nnabuhie 3M sales 104164068 0 from
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NETCONSTRUCT NIGERIA LIMITED to ASHAFA
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBRAHIM ABDULLAHI


In [42]:
# reset index in order to prevent problmems of indexation
master_df = pd.concat(finalized_dfs[:]).reset_index(drop=True)

In [43]:
# extra cleaning

master_df = master_df[master_df['Remarks']!= 'Remarks']
master_df = master_df[master_df['Remarks']!= 'Balance as at Last Transaction.']
master_df = master_df[master_df["Trans. Date"] != 'Trans. Date']

In [44]:
master_df = master_df.reset_index(drop=True)

In [45]:
transaction_not_null = master_df[~master_df["Trans. Date"].isna()]
index_list_of_transaction = list(transaction_not_null.index)

In [46]:
# all the transaction whithout a null starting date
transaction_not_null

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks
0,07-Dec-2020,07-Dec-2020,'0GAPS,NaN,"308,750.00","278,851.80",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS via GAPS
5,07-Dec-2020,07-Dec-2020,'0GAPS,NaN,"82,092.39","360,944.19",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS via GAPS
10,07-Dec-2020,07-Dec-2020.1,'0GWTR,"50,000.00",NaN,"310,944.19",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS Via GTWorld\rEneke ...
11,07-Dec-2020,07-Dec-2020,'0NIPG,"140,000.00",NaN,"170,944.19",E- CHANNELS,NIBSS Instant Payment Outward\r000013201207091...
12,07-Dec-2020,07-Dec-2020,'0NIPG,50.00,NaN,"170,894.19",E- CHANNELS,COMMISSION 000013201207091758000279938909\rNIP...
...,...,...,...,...,...,...,...,...
2265,08-Jun-2021,08-Jun-2021,'0NIPG,.75,NaN,"10,127.60",E- CHANNELS,VALUE ADDED TAX\r00001321060814452700017094897...
2266,09-Jun-2021,09-Jun-2021,'0USAT,"1,000.00",NaN,"9,127.60",E- CHANNELS,Airtime Purchase GTWORLD_QL-\r101CT00000000026...
2267,09-Jun-2021,09-Jun-2021,'0NIPG,"4,000.00",NaN,"5,127.60",E- CHANNELS,NIBSS Instant Payment Outward\r000013210609142...
2268,09-Jun-2021,09-Jun-2021,'0NIPG,10.00,NaN,"5,117.60",E- CHANNELS,COMMISSION 000013210609142049000172467995\rNIP...


In [47]:
# check that the column names is OK
transaction_not_null.columns

Index(['Trans. Date', 'Value. Date', 'Reference', 'Debits', 'Credits',
       'Balance', 'Originating Branch', 'Remarks'],
      dtype='object')

In [48]:
# create a dictionary with row indexes to keep
descr = {}

In [49]:
#transaction_not_null.loc[transaction_not_null.shape[0]]= [np.nan for i in range(8)]
# Create an arbitrary date (year 4021)
transaction_not_null.loc[master_df.index.max() +1,'Trans. Date'] = '01-Apr-4021'
#transaction_not_null = transaction_not_null[transaction_not_null["Trans. Date"] != 'Trans. Date']

<ipython-input-49-38623a337663>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_not_null.loc[master_df.index.max() +1,'Trans. Date'] = '01-Apr-4021'


In [50]:
# visualize the dataframe and its modification
transaction_not_null 

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks
0,07-Dec-2020,07-Dec-2020,'0GAPS,NaN,"308,750.00","278,851.80",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS via GAPS
5,07-Dec-2020,07-Dec-2020,'0GAPS,NaN,"82,092.39","360,944.19",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS via GAPS
10,07-Dec-2020,07-Dec-2020.1,'0GWTR,"50,000.00",NaN,"310,944.19",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS Via GTWorld\rEneke ...
11,07-Dec-2020,07-Dec-2020,'0NIPG,"140,000.00",NaN,"170,944.19",E- CHANNELS,NIBSS Instant Payment Outward\r000013201207091...
12,07-Dec-2020,07-Dec-2020,'0NIPG,50.00,NaN,"170,894.19",E- CHANNELS,COMMISSION 000013201207091758000279938909\rNIP...
...,...,...,...,...,...,...,...,...
2266,09-Jun-2021,09-Jun-2021,'0USAT,"1,000.00",NaN,"9,127.60",E- CHANNELS,Airtime Purchase GTWORLD_QL-\r101CT00000000026...
2267,09-Jun-2021,09-Jun-2021,'0NIPG,"4,000.00",NaN,"5,127.60",E- CHANNELS,NIBSS Instant Payment Outward\r000013210609142...
2268,09-Jun-2021,09-Jun-2021,'0NIPG,10.00,NaN,"5,117.60",E- CHANNELS,COMMISSION 000013210609142049000172467995\rNIP...
2269,09-Jun-2021,09-Jun-2021,'0NIPG,.75,NaN,"5,116.85",E- CHANNELS,VALUE ADDED TAX\r00001321060914204900017246799...


In [51]:
# all the indexes of the transaction with dates
index_with_dates = transaction_not_null.index

In [52]:
descr ={}
for step in index_with_dates:
    descr[str(step)] = []
descr

{'0': [],
 '5': [],
 '10': [],
 '11': [],
 '12': [],
 '13': [],
 '14': [],
 '15': [],
 '16': [],
 '18': [],
 '19': [],
 '20': [],
 '21': [],
 '22': [],
 '23': [],
 '24': [],
 '25': [],
 '27': [],
 '28': [],
 '29': [],
 '30': [],
 '31': [],
 '32': [],
 '33': [],
 '37': [],
 '42': [],
 '48': [],
 '55': [],
 '59': [],
 '64': [],
 '70': [],
 '74': [],
 '75': [],
 '76': [],
 '77': [],
 '78': [],
 '79': [],
 '80': [],
 '81': [],
 '82': [],
 '83': [],
 '84': [],
 '85': [],
 '86': [],
 '87': [],
 '88': [],
 '89': [],
 '90': [],
 '92': [],
 '93': [],
 '94': [],
 '95': [],
 '96': [],
 '97': [],
 '98': [],
 '99': [],
 '100': [],
 '104': [],
 '109': [],
 '115': [],
 '122': [],
 '125': [],
 '130': [],
 '136': [],
 '141': [],
 '142': [],
 '143': [],
 '144': [],
 '145': [],
 '146': [],
 '147': [],
 '149': [],
 '150': [],
 '151': [],
 '152': [],
 '153': [],
 '154': [],
 '155': [],
 '156': [],
 '158': [],
 '159': [],
 '160': [],
 '161': [],
 '162': [],
 '163': [],
 '164': [],
 '165': [],
 '166': [],
 '

In [53]:
step = 0    

for idx, step in enumerate(index_with_dates):
    if idx < len(index_with_dates)-1:
        for ind in range(index_with_dates[idx], index_with_dates[idx+1], 1):
            if ind < index_with_dates[idx+1]: 
                if str(master_df.loc[ind, 'Remarks']) != 'nan':
                    descr[str(step)] += [str(master_df.loc[ind, 'Remarks'])]
    else:
        for ind in range(index_with_dates[idx], master_df.shape[0], 1):
            if str(master_df.loc[ind, 'Remarks']) != 'nan':
                descr[str(step)] += [str(master_df.loc[ind, 'Remarks'])]

In [54]:
for key in descr.keys():
    descr[key] = (''.join(descr[key])).replace('\r',' ')

In [55]:
descr

{'0': 'TRANSFER BETWEEN CUSTOMERS via GAPScom on Babawuro A 13M sales 104164162 0 fromNETCONSTRUCT NIGERIA LIMITED to ASHAFAIBRAHIM ABDULLAHIREF: 0005628626104164162000000',
 '5': 'TRANSFER BETWEEN CUSTOMERS via GAPScom on Ekene Nnabuhie 3M sales 104164068 0 fromNETCONSTRUCT NIGERIA LIMITED to ASHAFAIBRAHIM ABDULLAHIREF: 0005628626104164068000000',
 '10': 'TRANSFER BETWEEN CUSTOMERS Via GTWorld Eneke   REF:681976569000000500002012070913 from ASHAFA IBRAHIM ABDULLAHI  to  SALAMI, ABUBAKAR',
 '11': 'NIBSS Instant Payment Outward 000013201207091758000279938909 via GTWORLD Babawuro Ali TO FBN/MANCHA GILBERT SIMI /53.75/REF:GW6819765690000014000020120709  f',
 '12': 'COMMISSION 000013201207091758000279938909 NIP TRANSFER COMMISSION FOR 000013201207091758000279938909 via GTWORLD Babawuro Ali TO FBN/MANCHA GILBERT SIMI ReF:GW681976569000001400002012070916',
 '13': 'VALUE ADDED TAX 000013201207091758000279938909 VAT ON NIP TRANSFER FOR 000013201207091758000279938909 via GTWORLD Babawuro Ali TO

In [56]:
# Dataframe of the transactions
pd.DataFrame.from_dict(descr,  orient='index', columns=['remarks_one_line'])

,remarks_one_line
0,TRANSFER BETWEEN CUSTOMERS via GAPScom on Baba...
5,TRANSFER BETWEEN CUSTOMERS via GAPScom on Eken...
10,TRANSFER BETWEEN CUSTOMERS Via GTWorld Eneke ...
11,NIBSS Instant Payment Outward 0000132012070917...
12,COMMISSION 000013201207091758000279938909 NIP ...
...,...
2266,Airtime Purchase GTWORLD_QL- 101CT000000000265...
2267,NIBSS Instant Payment Outward 0000132106091420...
2268,COMMISSION 000013210609142049000172467995 NIP ...
2269,VALUE ADDED TAX 000013210609142049000172467995...


In [1]:
import json

In [12]:
my_j =  "{\"Transactions\": [{\"transactionType\": \"SALES\",\"transactionAmount\": 10.05}]}"

In [13]:
json.loads(my_j)

{'Transactions': [{'transactionType': 'SALES', 'transactionAmount': 10.05}]}

In [6]:
my_j['msg'].keys()

AttributeError: 'str' object has no attribute 'keys'

In [ ]:
# trial on 1 dataframe

In [217]:
first_transaction = df[1]

first_transaction_not_null = first_transaction[~first_transaction["Trans. Date"].isna()]
index_list_of_transaction = list(first_transaction_not_null.index)
first_transaction_not_null.columns

Index(['Trans. Date', 'Value. Date', 'Reference', 'Debits', 'Credits',
       'Balance', 'Originating Branch', 'Remarks'],
      dtype='object')

In [110]:
first_transaction

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks
0,01-Apr-2021,01-Apr-2021,'0NIPG,"6,180.00",NaN,118.78,E- CHANNELS,NIBSS Instant Payment Outward
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000013210401074110000065685369 via
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTWORLD TO LAGASEF VENTURES LTD
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/26.875/REF:GW2249121800000000618021040107
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
5,01-Apr-2021,01-Apr-2021,'0NIPG,25.00,NaN,93.78,E- CHANNELS,COMMISSION 000013210401074110000065685369
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIP TRANSFER COMMISSION FOR
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000013210401074110000065685369 via
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTWORLD TO LAGASEF VENTURES LTD
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ReF:GW224912180000000061802104010739


In [111]:
descr = {}
first_transaction.shape

(10, 8)

In [112]:
first_transaction.loc[first_transaction.shape[0]]= [np.nan for i in range(8)]
first_transaction.loc[first_transaction.shape[0],'Trans. Date'] = '01-Apr-4021'

In [113]:
first_transaction

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks
0,01-Apr-2021,01-Apr-2021,'0NIPG,"6,180.00",NaN,118.78,E- CHANNELS,NIBSS Instant Payment Outward
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000013210401074110000065685369 via
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTWORLD TO LAGASEF VENTURES LTD
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/26.875/REF:GW2249121800000000618021040107
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
5,01-Apr-2021,01-Apr-2021,'0NIPG,25.00,NaN,93.78,E- CHANNELS,COMMISSION 000013210401074110000065685369
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIP TRANSFER COMMISSION FOR
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000013210401074110000065685369 via
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTWORLD TO LAGASEF VENTURES LTD
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ReF:GW224912180000000061802104010739


In [205]:
index_with_dates = list(first_transaction[~first_transaction['Trans. Date'].isnull()].index)

In [206]:
descr ={}
for step in index_with_dates:
    descr[step] = []
descr

{0: [], 5: [], 11: []}

In [207]:
index_with_dates

[0, 5, 11]

In [211]:

step = 0    

for idx, step in enumerate(index_with_dates):
    print(idx)
    if idx < len(index_with_dates)-1:
        for ind in range(index_with_dates[idx], index_with_dates[idx+1], 1):
            if ind < index_with_dates[idx+1]: 
                if str(first_transaction.loc[ind, 'Remarks']) != 'nan':
                    descr[step] += [str(first_transaction.loc[ind, 'Remarks'])]
    else:
        for ind in range(index_with_dates[idx], first_transaction.shape[0], 1):
            if str(first_transaction.loc[ind, 'Remarks']) != 'nan':
                descr[step] += [str(first_transaction.loc[ind, 'Remarks'])]

0
1
2


In [214]:
for key in descr.keys():
    descr[key] = (''.join(descr[key]))

In [215]:
descr

{0: 'NIBSS Instant Payment Outward000013210401074110000065685369 viaGTWORLD  TO LAGASEF VENTURES LTD/26.875/REF:GW2249121800000000618021040107f',
 5: 'COMMISSION 000013210401074110000065685369NIP TRANSFER COMMISSION FOR000013210401074110000065685369 viaGTWORLD  TO LAGASEF VENTURES LTDReF:GW224912180000000061802104010739',
 11: ''}

# 2. Data conversion

In [395]:
import time
import typing

In [396]:
def time_my_func(func):
    
    def inner_func(a_string):
        start = time.time()
        a =  func(a_string)
        end = time.time()
        print(f"it took : {(end-start)*1e6} microseconds")
        return a
    return inner_func
        

In [400]:
@time_my_func
def process(my_string:str) -> int:
    return(len(my_string))

In [401]:
process("this sheez is crazy")

it took : 0.95367431640625 microseconds


19

In [365]:
def parameterize(c):
    def smart_div(func):
        def inner_f(a,b):
            return(func(c,a))
        return inner_f
    return smart_div

In [366]:
@parameterize(500)
def div(a,b):
    return a/b

In [367]:
div(3,4)

166.66666666666666

In [351]:

div(30,4)

SyntaxError: invalid syntax (<ipython-input-351-605352f616b9>, line 2)

# 3. Conversion evaluation